# Data Cleaning

## Introduction

This notebook goes through a necessary step of any data science project - data cleaning. Data cleaning is a time consuming and unenjoyable task, yet it's a very important one. Keep in mind, "garbage in, garbage out". Feeding dirty data into a model will give us results that are meaningless.

Specifically, we'll be walking through:

1. **Getting the data - **in this case, we'll be scraping data from a website
2. **Cleaning the data - **we will walk through popular text pre-processing techniques
3. **Organizing the data - **we will organize the cleaned data into a way that is easy to input into other algorithms

The output of this notebook will be clean, organized data in two standard text formats:

1. **Corpus** - a collection of text
2. **Document-Term Matrix** - word counts in matrix format

## Problem Statement

As a reminder, our goal is to look at transcripts of various comedians and note their similarities and differences. Specifically, I'd like to know if Ali Wong's comedy style is different than other comedians, since she's the comedian that got me interested in stand up comedy.

## Getting The Data

Luckily, there are wonderful people online that keep track of stand up routine transcripts. [Scraps From The Loft](http://scrapsfromtheloft.com) makes them available for non-profit and educational purposes.

To decide which comedians to look into, I went on IMDB and looked specifically at comedy specials that were released in the past 5 years. To narrow it down further, I looked only at those with greater than a 7.5/10 rating and more than 2000 votes. If a comedian had multiple specials that fit those requirements, I would pick the most highly rated one. I ended up with a dozen comedy specials.

In [4]:
# Web scraping, pickle imports
import requests
from bs4 import BeautifulSoup
import pickle

# Scrapes transcript data from scrapsfromtheloft.com
def url_to_transcript(url):
    '''Returns transcript data specifically from scrapsfromtheloft.com.'''
    page = requests.get(url).text
    soup = BeautifulSoup(page, "lxml")
    text = [p.text for p in soup.find(class_="transcript-text").find_all('p')]
    print(url)
    return text


# URLs of transcripts in scope
urls = ['https://scrapsfromtheloft.com/comedy/jon-stewart-interviews-george-carlin-transcript/',
        'https://scrapsfromtheloft.com/comedy/kevin-james-irregardless-transcript/',
        'https://scrapsfromtheloft.com/comedy/pete-davidson-turbo-fonzarelli-transcript/',
        'https://scrapsfromtheloft.com/comedy/matt-rife-matthew-steven-rife-transcript/',
        'https://scrapsfromtheloft.com/comedy/david-nihill-cultural-appreciation-transcript/',
        'https://scrapsfromtheloft.com/comedy/shane-gillis-live-in-austin-transcript/',
        'https://scrapsfromtheloft.com/comedy/lewis-black-tragically-i-need-you-transcript/',
        'https://scrapsfromtheloft.com/comedy/stavros-halkias-live-at-the-lodge-room-transcript/',
        'https://scrapsfromtheloft.com/comedy/george-carlin-im-glad-im-dead-transcript/',
        'https://scrapsfromtheloft.com/comedy/leanne-morgan-im-every-woman-transcript/',
        'https://scrapsfromtheloft.com/comedy/louis-ck-at-the-dolby-transcript/',
        'https://scrapsfromtheloft.com/comedy/roseanne-barr-cancel-this-transcript/',
        'https://scrapsfromtheloft.com/comedy/sammy-obeid-where-is-bethlehem-transcript/',
        'https://scrapsfromtheloft.com/comedy/gabriel-iglesias-stadium-fluffy-transcript/',
        'https://scrapsfromtheloft.com/comedy/kate-berlant-cinnamon-in-the-wind-transcript/'
      ]

# Comedian names
comedians = ['jon', 'kevin', 'pete', 'matt', 'david', 'shane', 'lewis', 'stavros', 'george', 'leanne', 'louis', 'roseanne','sammy','gabriel','kate']


In [5]:
# # Actually request transcripts (takes a few minutes to run)
transcripts_eval = [url_to_transcript(u) for u in urls]

AttributeError: 'NoneType' object has no attribute 'find_all'

In [6]:
# # Pickle files for later use

# # Make a new directory to hold the text files
import os
import pickle

# Create directory if it doesn't exist
if not os.path.exists("transcripts_eval"):
    os.mkdir("transcripts_eval")

# Save transcripts into individual files
for i, c in enumerate(comedians):
    with open("transcripts_eval/" + c + ".txt", "wb") as file:
        pickle.dump(transcripts_eval[i], file)


In [7]:
# Load pickled files
data = {}
for i, c in enumerate(comedians):
    with open("transcripts_eval/" + c + ".txt", "rb") as file:
        data[c] = pickle.load(file)

In [8]:
# Double check to make sure data has been loaded properly
data.keys()

dict_keys(['jon', 'kevin', 'pete', 'matt', 'david', 'shane', 'lewis', 'stavros', 'george', 'leanne', 'louis', 'roseanne', 'sammy', 'gabriel', 'kate'])

In [9]:
# More checks
data['jon'][:2]

['In an interview conducted by Jon Stewart, George Carlin talks about various aspects of his life and career, reflecting upon his childhood, formative influences, and his perspectives on comedy and creativity.',
 'From a young age, Carlin felt an innate inclination toward the world of performance and the art of eliciting laughter from others. He attributes much of his comedic beginnings to the nurturing environment provided by his parents, individuals who exuded both humor and creativity. Additionally, Carlin highlights the impact of his grandfather, who possessed a dual identity as a law enforcement officer and a writer, and his mother, who held a prominent role as an advertising executive in his life.']

## Cleaning The Data

When dealing with numerical data, data cleaning often involves removing null values and duplicate data, dealing with outliers, etc. With text data, there are some common data cleaning techniques, which are also known as text pre-processing techniques.

With text data, this cleaning process can go on forever. There's always an exception to every cleaning step. So, we're going to follow the MVP (minimum viable product) approach - start simple and iterate. Here are a bunch of things you can do to clean your data. We're going to execute just the common cleaning steps here and the rest can be done at a later point to improve our results.

**Common data cleaning steps on all text:**
* Make text all lower case
* Remove punctuation
* Remove numerical values
* Remove common non-sensical text (/n)
* Tokenize text
* Remove stop words

**More data cleaning steps after tokenization:**
* Stemming / lemmatization
* Parts of speech tagging
* Create bi-grams or tri-grams
* Deal with typos
* And more...

In [10]:
# Let's take a look at our data again
next(iter(data.keys()))

'jon'

In [11]:
# Notice that our dictionary is currently in key: comedian, value: list of text format
next(iter(data.values()))

['In an interview conducted by Jon Stewart, George Carlin talks about various aspects of his life and career, reflecting upon his childhood, formative influences, and his perspectives on comedy and creativity.',
 'From a young age, Carlin felt an innate inclination toward the world of performance and the art of eliciting laughter from others. He attributes much of his comedic beginnings to the nurturing environment provided by his parents, individuals who exuded both humor and creativity. Additionally, Carlin highlights the impact of his grandfather, who possessed a dual identity as a law enforcement officer and a writer, and his mother, who held a prominent role as an advertising executive in his life.',
 'Carlin’s journey through the conventional educational system was marked by a sense of not quite fitting in. He attended an experimental school that boldly discarded the typical grading system and eschewed corporal punishment, an environment that proved instrumental in fostering and 

In [12]:
# We are going to change this to key: comedian, value: string format
def combine_text(list_of_text):
    '''Takes a list of text and combines them into one large chunk of text.'''
    combined_text = ' '.join(list_of_text)
    return combined_text

In [13]:
# Combine it!
data_combined = {key: [combine_text(value)] for (key, value) in data.items()}

In [14]:
# We can either keep it in dictionary format or put it into a pandas dataframe
import pandas as pd
pd.set_option('max_colwidth',150)

data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns = ['transcript']
data_df = data_df.sort_index()
data_df

,transcript
david,"In this 2023 comedy special, David Nihill humorously navigates the complexities of identity, cultural norms, and immigration. The routine starts w..."
gabriel,[man] Can you please state your name? Martin Moreno. But you might know me as… Martinnnnn! I’ve been touring with Gabriel Iglesias for 20-plus yea...
george,George Carlin: I’m Glad I’m Dead (2024) is a controversial project that sparked ethical debate and concerns over the posthumous use of a comedian’...
jon,"In an interview conducted by Jon Stewart, George Carlin talks about various aspects of his life and career, reflecting upon his childhood, formati..."
kate,"Whoa! Okay, yeah. Good. Okay, don’t embarrass yourself, okay. Ohh, the expectations. Crushing, I would argue. Absolutely crushing, debilitating in..."
kevin,"Kevin James: Irregardless (2024) In Kevin James: Irregardless, a 2024 stand-up comedy special, Kevin James delivers a performance that expertly bl..."
leanne,"Leanne Morgan: I’m Every Woman (2023) In “I’m Every Woman,” Leanne Morgan delivers a stand-up performance that blends self-deprecating humor with ..."
lewis,"Lewis Black: Tragically, I Need You (2023) is a stand-up comedy show where Lewis Black, known for his signature mix of sarcasm and outrage, reflec..."
louis,"“Louis C.K.: At The Dolby” is Louis C.K.’s third self-released stand-up comedy special since the pandemic, following his previous Grammy-winning a..."
matt,"In his second hour-long comedy special, “Matthew Stephen Rife,” Matthew Rife presents a candid, often provocative performance, unrestrained by net..."


In [15]:
# Let's take a look at the transcript for david
data_df.transcript.loc['david']

'In this 2023 comedy special, David Nihill humorously navigates the complexities of identity, cultural norms, and immigration. The routine starts with an amusing encounter at an immigration checkpoint, highlighting Nihill’s Irish heritage and the officer’s misplaced connection to it. Nihill then adopts a Latino identity as a comedic device to critique automated customer service systems. He cleverly connects Irish and Latino cultures, drawing parallels in their experiences, particularly around issues like immigration and religion, using humor to highlight these similarities. Nihill also addresses the evolving perception of Irish immigrants in America, blending history with humor. He reflects on the Irish journey from being a marginalized group to gaining a more celebrated status in American society. His narrative includes witty anecdotes about his family’s history and their unique assimilation into American culture, adding a personal touch to broader discussions on race and identity. Th

In [16]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [17]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_df.transcript.apply(round1))
data_clean

,transcript
david,in this comedy special david nihill humorously navigates the complexities of identity cultural norms and immigration the routine starts with an a...
gabriel,can you please state your name martin moreno but you might know me as… martinnnnn i’ve been touring with gabriel iglesias for years martinnnnn a...
george,george carlin i’m glad i’m dead is a controversial project that sparked ethical debate and concerns over the posthumous use of a comedian’s perso...
jon,in an interview conducted by jon stewart george carlin talks about various aspects of his life and career reflecting upon his childhood formative ...
kate,whoa okay yeah good okay don’t embarrass yourself okay ohh the expectations crushing i would argue absolutely crushing debilitating in every way i...
kevin,kevin james irregardless in kevin james irregardless a standup comedy special kevin james delivers a performance that expertly blends humor with...
leanne,leanne morgan i’m every woman in “i’m every woman” leanne morgan delivers a standup performance that blends selfdeprecating humor with relatable ...
lewis,lewis black tragically i need you is a standup comedy show where lewis black known for his signature mix of sarcasm and outrage reflects on the i...
louis,“louis ck at the dolby” is louis ck’s third selfreleased standup comedy special since the pandemic following his previous grammywinning and nomina...
matt,in his second hourlong comedy special “matthew stephen rife” matthew rife presents a candid often provocative performance unrestrained by network ...


In [18]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [19]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean.transcript.apply(round2))
data_clean

,transcript
david,in this comedy special david nihill humorously navigates the complexities of identity cultural norms and immigration the routine starts with an a...
gabriel,can you please state your name martin moreno but you might know me as martinnnnn ive been touring with gabriel iglesias for years martinnnnn and...
george,george carlin im glad im dead is a controversial project that sparked ethical debate and concerns over the posthumous use of a comedians persona ...
jon,in an interview conducted by jon stewart george carlin talks about various aspects of his life and career reflecting upon his childhood formative ...
kate,whoa okay yeah good okay dont embarrass yourself okay ohh the expectations crushing i would argue absolutely crushing debilitating in every way im...
kevin,kevin james irregardless in kevin james irregardless a standup comedy special kevin james delivers a performance that expertly blends humor with...
leanne,leanne morgan im every woman in im every woman leanne morgan delivers a standup performance that blends selfdeprecating humor with relatable anec...
lewis,lewis black tragically i need you is a standup comedy show where lewis black known for his signature mix of sarcasm and outrage reflects on the i...
louis,louis ck at the dolby is louis cks third selfreleased standup comedy special since the pandemic following his previous grammywinning and nominated...
matt,in his second hourlong comedy special matthew stephen rife matthew rife presents a candid often provocative performance unrestrained by network or...


**NOTE:** This data cleaning aka text pre-processing step could go on for a while, but we are going to stop for now. After going through some analysis techniques, if you see that the results don't make sense or could be improved, you can come back and make more edits such as:
* Mark 'cheering' and 'cheer' as the same word (stemming / lemmatization)
* Combine 'thank you' into one term (bi-grams)
* And a lot more...

## Organizing The Data

I mentioned earlier that the output of this notebook will be clean, organized data in two standard text formats:
1. **Corpus - **a collection of text
2. **Document-Term Matrix - **word counts in matrix format

### Corpus

We already created a corpus in an earlier step. The definition of a corpus is a collection of texts, and they are all put together neatly in a pandas dataframe here.

In [20]:
# Let's take a look at our dataframe
data_df

,transcript
david,"In this 2023 comedy special, David Nihill humorously navigates the complexities of identity, cultural norms, and immigration. The routine starts w..."
gabriel,[man] Can you please state your name? Martin Moreno. But you might know me as… Martinnnnn! I’ve been touring with Gabriel Iglesias for 20-plus yea...
george,George Carlin: I’m Glad I’m Dead (2024) is a controversial project that sparked ethical debate and concerns over the posthumous use of a comedian’...
jon,"In an interview conducted by Jon Stewart, George Carlin talks about various aspects of his life and career, reflecting upon his childhood, formati..."
kate,"Whoa! Okay, yeah. Good. Okay, don’t embarrass yourself, okay. Ohh, the expectations. Crushing, I would argue. Absolutely crushing, debilitating in..."
kevin,"Kevin James: Irregardless (2024) In Kevin James: Irregardless, a 2024 stand-up comedy special, Kevin James delivers a performance that expertly bl..."
leanne,"Leanne Morgan: I’m Every Woman (2023) In “I’m Every Woman,” Leanne Morgan delivers a stand-up performance that blends self-deprecating humor with ..."
lewis,"Lewis Black: Tragically, I Need You (2023) is a stand-up comedy show where Lewis Black, known for his signature mix of sarcasm and outrage, reflec..."
louis,"“Louis C.K.: At The Dolby” is Louis C.K.’s third self-released stand-up comedy special since the pandemic, following his previous Grammy-winning a..."
matt,"In his second hour-long comedy special, “Matthew Stephen Rife,” Matthew Rife presents a candid, often provocative performance, unrestrained by net..."


In [3]:
# Let's add the comedians' full names as well
full_names = [
    'Jon Stewart',
    'Kevin Hart',
    'Pete Davidson',
    'Matt Damon',  
    'David Letterman',
    'Shane Gillis',
    'Lewis Black',
    'Stavros Halkias',
    'George Carlin',
    'Leanne Morgan',
    'Louis C.K.',
    'Roseanne Barr',
    'Sammy Davis Jr.',  
    'Gabriel Iglesias',
    'Kate McKinnon'
]
data_df['full_name'] = full_names
data_df

NameError: name 'data_df' is not defined

In [21]:
# Let's pickle it for later use
data_df.to_pickle("corpus.pkl")

### Document-Term Matrix

For many of the techniques we'll be using in future notebooks, the text must be tokenized, meaning broken down into smaller pieces. The most common tokenization technique is to break down text into words. We can do this using scikit-learn's CountVectorizer, where every row will represent a different document and every column will represent a different word.

In addition, with CountVectorizer, we can remove stop words. Stop words are common words that add no additional meaning to text such as 'a', 'the', etc.

In [25]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.transcript)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names_out())
data_dtm.index = data_clean.index
data_dtm


,aaaaah,aaaahhh,aaah,aah,abandoned,abc,abilities,ability,able,abnormal,...,zone,zones,zoo,zoom,zorro,zuck,zuckerbergs,álvarez,ándale,ñañaras
david,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3,0,0,0,0,0
gabriel,0,0,1,0,0,1,0,0,0,0,...,0,1,0,4,0,0,0,3,1,1
george,0,0,0,0,0,0,0,1,3,0,...,0,0,0,0,0,1,1,0,0,0
jon,0,0,0,0,0,0,0,0,2,0,...,1,0,0,0,0,0,0,0,0,0
kate,0,0,0,1,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
kevin,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
leanne,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
lewis,0,0,0,0,0,0,0,0,1,4,...,0,0,0,0,0,0,0,0,0,0
louis,0,1,0,4,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
matt,0,0,0,0,0,0,0,0,3,0,...,1,0,0,0,0,0,0,0,0,0


In [26]:
# Let's pickle it for later use
data_dtm.to_pickle("dtm.pkl")

In [27]:
# Let's also pickle the cleaned data (before we put it in document-term matrix format) and the CountVectorizer object
data_clean.to_pickle('data_clean.pkl')
pickle.dump(cv, open("cv.pkl", "wb"))

## Additional Exercises

1. Can you add an additional regular expression to the clean_text_round2 function to further clean the text?
2. Play around with CountVectorizer's parameters. What is ngram_range? What is min_df and max_df?